# 🎯 Exercise 1: Build Your First Multi-Tool Agent

**Difficulty:** Beginner  
**Estimated Time:** 45-60 minutes

## Task
Create an agent with three custom tools that decide which one to use based on the user's query.

## Tools to Build
1. **Weather Tool** - Returns simulated weather for a given city
2. **Dictionary Tool** - Looks up word definitions  
3. **Web Search Tool** - Uses DuckDuckGo to search the web for information

## Requirements
- Define tools using `@tool` decorator
- Bind tools to LLM
- Implement conditional routing (agent decides which tool to use)
- Handle cases where no tool is needed
- Use DuckDuckGo search library

## Example Interactions
```
Query: "What's the weather in Lagos?"
Decision: Uses weather tool → Returns weather data

Query: "Define the word 'ephemeral'"
Decision: Uses dictionary tool → Returns definition

Query: "Search for latest AI news"
Decision: Uses web search tool → Returns search results

Query: "What's the capital of France?"
Decision: No tool needed → Answers from knowledge
```

---

In [1]:
# Install required packages
%pip install -q python-dotenv langchain-openai langgraph duckduckgo-search

print("✅ Packages installed")

Note: you may need to restart the kernel to use updated packages.
✅ Packages installed


In [2]:
# Imports
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Literal, Sequence
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
import operator
import os
from duckduckgo_search import DDGS

print("✅ All imports successful")

✅ All imports successful


In [3]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

✅ API key loaded successfully


In [4]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


### Weather Tool

In [5]:
@tool
def get_weather(city: str) -> str:
    """
    Get the weather for a specific city.
    
    Use this tool when the user asks about weather conditions in a city.
    
    Args:
        city: The name of the city to get weather for
        
    Returns:
        Weather description for the location
    """
    weather_database = {
        "lagos": "Sunny, 28°C, Humidity: 75%, Light breeze from the southwest",
        "london": "Cloudy, 12°C, Humidity: 65%, Strong wind from the north",
        "new york": "Rainy, 15°C, Humidity: 80%, Heavy rain expected",
        "tokyo": "Clear, 22°C, Humidity: 55%, Gentle breeze",
        "paris": "Partly cloudy, 14°C, Humidity: 70%, Light drizzle",
        "sydney": "Sunny, 25°C, Humidity: 60%, Moderate wind",
    }
    
    city_lower = city.lower().strip()
    
    if city_lower in weather_database:
        return f"Weather in {city}: {weather_database[city_lower]}"
    else:
        return f"Weather data for '{city}' not available. Try: Lagos, London, New York, Tokyo, Paris, or Sydney."

print("✅ Weather tool created")

✅ Weather tool created


In [6]:
# Test the weather tool
test_weather = get_weather.invoke({"city": "Lagos"})
print(f"Test: {test_weather}")

Test: Weather in Lagos: Sunny, 28°C, Humidity: 75%, Light breeze from the southwest


### Dictionary Tool

In [7]:
@tool
def define_word(word: str) -> str:
    """
    Look up the definition of a word in the dictionary.
    
    Use this tool when the user asks for a word definition or the meaning of a word.
    
    Args:
        word: The word to look up
        
    Returns:
        Definition of the word
    """
    # Simulated dictionary database
    dictionary = {
        "ephemeral": "Lasting for a very short time; transient. Example: The beauty of cherry blossoms is ephemeral.",
        "serendipity": "The occurrence of events by chance in a happy or beneficial way. Example: Finding that old photo was pure serendipity.",
        "eloquent": "Fluent, persuasive, and expressive in speaking or writing. Example: She gave an eloquent speech.",
        "paradigm": "A typical example or pattern of something; a model. Example: The scientific paradigm shifted with Einstein's theories.",
        "ubiquitous": "Present, appearing, or found everywhere. Example: Smartphones are ubiquitous in modern society.",
        "perspicacious": "Having keen insight, discernment, or understanding. Example: Her perspicacious analysis revealed the problem.",
        "languid": "Lacking energy, vigor, or vitality; slow and relaxed. Example: The languid pace of summer days.",
        "meticulous": "Showing great attention to detail; very careful and precise. Example: His meticulous work was flawless.",
    }
    
    word_lower = word.lower().strip()
    
    if word_lower in dictionary:
        return f"Definition of '{word}': {dictionary[word_lower]}"
    else:
        return f"'{word}' not found in dictionary. Try one of these: ephemeral, serendipity, eloquent, paradigm, ubiquitous, perspicacious, languid, meticulous"

print("✅ Dictionary tool created")

✅ Dictionary tool created


In [8]:
# Test the dictionary tool
test_definition = define_word.invoke({"word": "ephemeral"})
print(f"Test: {test_definition}")

Test: Definition of 'ephemeral': Lasting for a very short time; transient. Example: The beauty of cherry blossoms is ephemeral.


### Web Search Tool (DuckDuckGo)

In [9]:
@tool
def web_search(query: str) -> str:
    """
    Search the web for information using DuckDuckGo.
    
    Use this tool when the user asks for current information, news, or facts
    that require web search.
    
    Args:
        query: The search query
        
    Returns:
        Search results from DuckDuckGo
    """
    try:
        # Create DDGS instance and search
        results = DDGS().text(query, max_results=5)
        
        if not results:
            return "No search results found."
        
        # Format results
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result['title']}\n"
                f"   {result['body']}\n"
                f"   Source: {result['href']}"
            )
        
        return "\n\n".join(formatted_results)
    
    except Exception as e:
        return f"Error during web search: {str(e)}"

print("✅ Web search tool created")

✅ Web search tool created


In [10]:
# Test the web search tool (optional - may take a moment)
print("Testing web search tool...")
try:
    test_search = web_search.invoke({"query": "Python programming tips"})
    print(f"Test (first 300 chars): {test_search[:300]}...")
except Exception as e:
    print(f"Note: Web search test skipped - {e}")

Testing web search tool...


C:\Users\akind\AppData\Local\Temp\ipykernel_22608\627364244.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=5)


Test (first 300 chars): No search results found....


### Define Agent State

In [11]:
# Define state that accumulates messages
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

print("✅ Agent state defined")
print("   - Stores conversation messages")
print("   - Messages accumulate for conversation history")

✅ Agent state defined
   - Stores conversation messages
   - Messages accumulate for conversation history


### Bind Tools to LLM

In [12]:
# Create list of tools
tools = [get_weather, define_word, web_search]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print(f"✅ {len(tools)} tools bound to LLM")
print("   - get_weather")
print("   - define_word")
print("   - web_search")

✅ 3 tools bound to LLM
   - get_weather
   - define_word
   - web_search


### Create Agent Node

In [13]:
# System prompt
system_prompt = """You are a helpful assistant with access to several tools:

1. get_weather - Get weather for a city
2. define_word - Look up word definitions
3. web_search - Search the web for information

Use these tools when appropriate to answer user queries accurately.
For general knowledge questions that don't require tools, answer directly.
"""

def assistant(state: AgentState):
    """
    Agent node that decides whether to use tools or answer directly.
    """
    messages = [SystemMessage(content=system_prompt)] + list(state["messages"])
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}

print("✅ Assistant node created")

✅ Assistant node created


### Implement Conditional Routing

In [14]:
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """
    Determine if agent should call tools or finish.
    
    This is the KEY decision point:
    - If the agent's last message contains tool_calls → route to tools
    - Otherwise → end the conversation
    """
    last_message = state["messages"][-1]
    
    # Check if last message contains tool calls
    if last_message.tool_calls:
        print(f"   → Agent decided to use tool(s): {[tc['name'] for tc in last_message.tool_calls]}")
        return "tools"
    else:
        print(f"   → Agent decided to answer directly (no tools needed)")
        return "__end__"

print("✅ Conditional routing function created")

✅ Conditional routing function created


### Build the Graph

In [15]:
# Create StateGraph
builder = StateGraph(AgentState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")  # Start with assistant
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)  # Conditional: tools or end
builder.add_edge("tools", "assistant")  # After tools, back to assistant

# Compile graph
agent = builder.compile()

print("✅ Agent graph compiled")
print("\nGraph Architecture:")
print("  START → assistant → [decides]")
print("                         → tools → assistant → [decides again]")
print("                         → END")

✅ Agent graph compiled

Graph Architecture:
  START → assistant → [decides]
                         → tools → assistant → [decides again]
                         → END


### Test the Agent

In [16]:
def test_agent(query: str):
    """
    Test the agent with a query.
    """
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}")
    
    result = agent.invoke({"messages": [HumanMessage(content=query)]})
    
    # Extract and print response
    for msg in result["messages"]:
        if isinstance(msg, AIMessage) and not msg.tool_calls:
            print(f"\nResponse: {msg.content}")
    
    print(f"{'='*70}\n")

print("✅ Test function created")

✅ Test function created


In [17]:
test_agent("What's the weather in ogun?")


Query: What's the weather in ogun?
   → Agent decided to use tool(s): ['get_weather']
   → Agent decided to use tool(s): ['get_weather']
   → Agent decided to answer directly (no tools needed)

Response: The weather in Lagos, which is in the Ogun state area, is currently sunny with a temperature of 28°C. The humidity is at 75%, and there's a light breeze coming from the southwest.



### Dictionary Query (Should Use Dictionary Tool)

In [18]:
test_agent("Define the word 'python'")


Query: Define the word 'python'
   → Agent decided to use tool(s): ['define_word']
   → Agent decided to answer directly (no tools needed)

Response: The word "python" does not have a definition in the dictionary I accessed. However, I can provide some information:

1. **Python (animal)**: A type of large snake found in Africa, Asia, and Australia, known for constricting its prey. 
2. **Python (programming language)**: A high-level programming language known for its readability and simplicity, widely used in web development, data analysis, artificial intelligence, and more.

If you're looking for a specific definition or context, please let me know!



### Web Search Query (Should Use Web Search Tool)

In [19]:
test_agent("What are the latest developments in tax?")


Query: What are the latest developments in tax?
   → Agent decided to use tool(s): ['web_search']


C:\Users\akind\AppData\Local\Temp\ipykernel_22608\627364244.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=5)


   → Agent decided to use tool(s): ['web_search']


C:\Users\akind\AppData\Local\Temp\ipykernel_22608\627364244.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=5)


   → Agent decided to use tool(s): ['web_search']


C:\Users\akind\AppData\Local\Temp\ipykernel_22608\627364244.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=5)


   → Agent decided to answer directly (no tools needed)

Response: It seems that I'm unable to find specific recent news articles regarding the latest developments in tax for 2023 at this moment. However, I can summarize some general trends that have been observed in recent years:

1. **Digital Services Taxes**: Many countries have been implementing or considering digital services taxes aimed at large tech companies that benefit from local markets without paying adequate taxes.

2. **Tax Reforms**: Various nations are undergoing tax reforms to simplify their tax codes, enhance compliance, and increase revenue. This includes changes in corporate tax rates and personal income tax brackets.

3. **Climate-Related Tax Incentives**: As part of efforts to combat climate change, some governments are introducing tax credits and incentives for renewable energy investments and electric vehicles.

4. **International Tax Agreements**: There have been ongoing discussions and agreements on internatio

### General Knowledge (Should NOT Use Any Tool)

In [20]:
test_agent("What is the capital of Nigeria?")


Query: What is the capital of Nigeria?
   → Agent decided to answer directly (no tools needed)

Response: The capital of Nigeria is Abuja.



### Multiple Tool Possibility (Agent Decides Best One)

In [21]:
# This test uses multiple tools - agent decides which to use!
test_agent("Tell me the weather in Lagos and define the word 'ubiquitous'")


Query: Tell me the weather in Lagos and define the word 'ubiquitous'
   → Agent decided to use tool(s): ['get_weather', 'define_word']
   → Agent decided to answer directly (no tools needed)

Response: The weather in Lagos is sunny with a temperature of 28°C, humidity at 75%, and a light breeze coming from the southwest.

The word 'ubiquitous' is defined as being present, appearing, or found everywhere. For example, "Smartphones are ubiquitous in modern society."



### Unknown Word (Should Handle Gracefully)

In [22]:
test_agent("What does 'xyz123abc' mean?")


Query: What does 'xyz123abc' mean?
   → Agent decided to use tool(s): ['define_word']
   → Agent decided to answer directly (no tools needed)

Response: The term 'xyz123abc' does not have a recognized meaning in standard dictionaries. It appears to be a combination of letters and numbers, possibly used as a username, password, or identifier. If it has a specific context or usage you are referring to, please provide more details!

